# Speakhuman Benchmarks

Performance comparison between **humanize** and **speakhuman**.

Each benchmark runs 100,000 iterations with 3 function calls per iteration (300K total calls per function).

In [ ]:
%pip install speakhuman humanize

In [ ]:
import time
import datetime as dt
import humanize
import speakhuman

print(f"humanize version:    {humanize.__version__}")
print(f"speakhuman version:  {speakhuman.__version__}")

# Check if Rust extension is available
try:
    import speakhuman._speakhuman_rs
    HAS_RUST = True
    print("Rust extension:      loaded")
except ImportError:
    HAS_RUST = False
    print("Rust extension:      not available (pure Python mode)")

ITERATIONS = 100_000

In [ ]:
def bench_compare(name, h_fn, s_fn, args_list):
    """Benchmark one function across both libraries."""
    # Warmup
    for args in args_list:
        h_fn(*args)
        s_fn(*args)

    # Benchmark humanize
    h_start = time.perf_counter()
    for _ in range(ITERATIONS):
        for args in args_list:
            h_fn(*args)
    h_time = time.perf_counter() - h_start

    # Benchmark speakhuman
    s_start = time.perf_counter()
    for _ in range(ITERATIONS):
        for args in args_list:
            s_fn(*args)
    s_time = time.perf_counter() - s_start

    ratio = h_time / s_time if s_time > 0 else float('inf')
    calls = ITERATIONS * len(args_list)

    return {
        'name': name,
        'humanize_ms': h_time * 1000,
        'speakhuman_ms': s_time * 1000,
        'ratio': ratio,
        'humanize_ops': calls / h_time,
        'speakhuman_ops': calls / s_time,
    }

## humanize vs speakhuman

In [ ]:
benchmarks = [
    ("intcomma", humanize.intcomma, speakhuman.intcomma, [
        (1_000_000,), (1_234_567.25,), (10311,),
    ]),
    ("intword", humanize.intword, speakhuman.intword, [
        (1_000_000,), (1_200_000_000,), (8_100_000_000_000,),
    ]),
    ("ordinal", humanize.ordinal, speakhuman.ordinal, [
        (1,), (103,), (111,),
    ]),
    ("apnumber", humanize.apnumber, speakhuman.apnumber, [
        (0,), (5,), (10,),
    ]),
    ("scientific", humanize.scientific, speakhuman.scientific, [
        (1000,), (0.3,), (5_781_651_000,),
    ]),
    ("fractional", humanize.fractional, speakhuman.fractional, [
        (0.3,), (1.3,), (1/3,),
    ]),
    ("metric", humanize.metric, speakhuman.metric, [
        (1500, "V"), (2e8, "W"), (220e-6, "F"),
    ]),
    ("naturalsize", humanize.naturalsize, speakhuman.naturalsize, [
        (3_000_000,), (1024 * 31,), (3000,),
    ]),
    ("naturaldelta", humanize.naturaldelta, speakhuman.naturaldelta, [
        (dt.timedelta(days=7),), (dt.timedelta(seconds=30),), (dt.timedelta(days=500),),
    ]),
    ("precisedelta", humanize.precisedelta, speakhuman.precisedelta, [
        (dt.timedelta(seconds=3633, days=2),),
        (dt.timedelta(seconds=1),),
        (dt.timedelta(days=370, hours=4),),
    ]),
]

print(f"Running {ITERATIONS:,} iterations x 3 calls per function...\n")
results = []
for name, h_fn, s_fn, args in benchmarks:
    r = bench_compare(name, h_fn, s_fn, args)
    results.append(r)
    print(f"  {name:<16} done")

print("\nDone!")

## Results

In [ ]:
total_h = sum(r['humanize_ms'] for r in results)
total_s = sum(r['speakhuman_ms'] for r in results)

print(f"{'Function':<16} {'humanize':>12} {'speakhuman':>12} {'Ratio':>10}")
print(f"{'-'*16} {'-'*12} {'-'*12} {'-'*10}")

for r in results:
    ratio_str = f"{r['ratio']:.2f}x"
    print(
        f"{r['name']:<16} "
        f"{r['humanize_ms']:>9.1f} ms "
        f"{r['speakhuman_ms']:>9.1f} ms "
        f"{ratio_str:>10}"
    )

print(f"{'-'*16} {'-'*12} {'-'*12} {'-'*10}")
total_ratio = total_h / total_s if total_s > 0 else 0
print(
    f"{'TOTAL':<16} "
    f"{total_h:>9.1f} ms "
    f"{total_s:>9.1f} ms "
    f"{total_ratio:>8.2f}x"
)
print(f"\nRatio > 1.0 means speakhuman is faster, < 1.0 means humanize is faster")

## Visual Comparison

In [ ]:
BAR_WIDTH = 40
max_time = max(max(r['humanize_ms'], r['speakhuman_ms']) for r in results)

print(f"  Time (ms) — lower is better\n")

for r in results:
    h_len = max(1, int(r['humanize_ms'] / max_time * BAR_WIDTH))
    s_len = max(1, int(r['speakhuman_ms'] / max_time * BAR_WIDTH))

    print(f"  {r['name']}")
    print(f"    humanize    {'\u2593' * h_len} {r['humanize_ms']:.1f}ms")
    print(f"    speakhuman  {'\u2588' * s_len} {r['speakhuman_ms']:.1f}ms")
    print()

## Throughput (operations/sec)

In [ ]:
print(f"{'Function':<16} {'humanize ops/s':>16} {'speakhuman ops/s':>18}")
print(f"{'-'*16} {'-'*16} {'-'*18}")

for r in results:
    print(
        f"{r['name']:<16} "
        f"{r['humanize_ops']:>14,.0f}/s "
        f"{r['speakhuman_ops']:>16,.0f}/s"
    )

## Rust vs Pure Python (internal)

If the Rust extension is available, compare the Rust-accelerated path vs the pure-Python fallback within speakhuman itself.

In [ ]:
if not HAS_RUST:
    print("Rust extension not available — skipping internal comparison.")
else:
    from speakhuman._speakhuman_rs import (
        intcomma as rs_intcomma,
        ordinal as rs_ordinal,
        naturalsize as rs_naturalsize,
        scientific as rs_scientific,
        apnumber as rs_apnumber,
    )
    from speakhuman.number import (
        _py_intcomma, _py_ordinal, _py_scientific, _py_apnumber,
    )
    from speakhuman.filesize import _py_naturalsize

    internal_benchmarks = [
        ("intcomma", _py_intcomma, rs_intcomma, [(1_000_000,), (1_234_567.25,)]),
        ("ordinal", _py_ordinal, rs_ordinal, [(1,), (103,), (111,)]),
        ("naturalsize", _py_naturalsize, rs_naturalsize, [(3_000_000,), (1024*31,)]),
        ("scientific", _py_scientific, rs_scientific, [(1000,), (0.3,)]),
        ("apnumber", _py_apnumber, rs_apnumber, [(0,), (5,), (10,)]),
    ]

    print(f"{'Function':<16} {'Python':>12} {'Rust':>12} {'Ratio':>10}")
    print(f"{'-'*16} {'-'*12} {'-'*12} {'-'*10}")

    for name, py_fn, rs_fn, args_list in internal_benchmarks:
        # Python
        start = time.perf_counter()
        for _ in range(ITERATIONS):
            for args in args_list:
                py_fn(*args)
        py_time = (time.perf_counter() - start) * 1000

        # Rust
        start = time.perf_counter()
        for _ in range(ITERATIONS):
            for args in args_list:
                rs_fn(*args)
        rs_time = (time.perf_counter() - start) * 1000

        ratio = py_time / rs_time if rs_time > 0 else 0
        print(f"{name:<16} {py_time:>9.1f} ms {rs_time:>9.1f} ms {ratio:>8.2f}x")

    print(f"\nRatio > 1.0 means Rust is faster, < 1.0 means Python is faster")
    print(f"Note: PyO3 FFI overhead can make Rust slower for simple string operations")

## Correctness Check

Verify both libraries produce identical output.

In [ ]:
checks = [
    ("intcomma(1000000)", humanize.intcomma(1_000_000), speakhuman.intcomma(1_000_000)),
    ("ordinal(42)", humanize.ordinal(42), speakhuman.ordinal(42)),
    ("intword(1200000)", humanize.intword(1_200_000), speakhuman.intword(1_200_000)),
    ("apnumber(5)", humanize.apnumber(5), speakhuman.apnumber(5)),
    ("naturalsize(1048576)", humanize.naturalsize(1_048_576), speakhuman.naturalsize(1_048_576)),
    ("scientific(2700)", humanize.scientific(2700), speakhuman.scientific(2700)),
    ("fractional(1.5)", humanize.fractional(1.5), speakhuman.fractional(1.5)),
    ("metric(1500, 'g')", humanize.metric(1500, 'g'), speakhuman.metric(1500, 'g')),
    ("naturaldelta(3h)", humanize.naturaldelta(dt.timedelta(hours=3)), speakhuman.naturaldelta(dt.timedelta(hours=3))),
]

all_match = True
print(f"{'Call':<28} {'humanize':>16} {'speakhuman':>16} {'Match':>7}")
print(f"{'-'*28} {'-'*16} {'-'*16} {'-'*7}")
for call, h_result, s_result in checks:
    match = str(h_result) == str(s_result)
    if not match:
        all_match = False
    print(f"{call:<28} {str(h_result):>16} {str(s_result):>16} {'yes' if match else 'NO':>7}")

print()
if all_match:
    print("All outputs match! speakhuman is a drop-in replacement for humanize.")
else:
    print("Some outputs differ — check the results above.")